# Extract

In [1]:
import pandas as pd
active_corporations=pd.read_csv(r'Active_Corporations___Beginning_1800.csv')
unemployment=pd.read_csv(r'Local_Area_Unemployment_Statistics__Beginning_1976.csv')
counties=pd.read_csv(r'counties.csv')

c:\users\simay\code\bigdata\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Transform

## Cleaning corporations dataset


In [2]:
not_null_date=active_corporations['Initial DOS Filing Date'].isnull().map(lambda x: not x)
active_corporations=active_corporations[not_null_date]
active_corporations['month']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[0]))
active_corporations['day']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[1]))
active_corporations['year']=active_corporations['Initial DOS Filing Date'].map(lambda x: int(x.split('/')[2]))
active_corporations=active_corporations[active_corporations['year'].map(lambda x: x>1976)]

In [3]:
def format_county(s):
    suffix=' County'
    if s.endswith(suffix):
        return s[:-len(suffix)].lower()
    return s.lower()
county_names=counties['County']
county_names=set(map(format_county,county_names))

In [4]:
not_null_county=active_corporations['County'].isnull().map(lambda x: not x)
active_corporations=active_corporations[not_null_county]
active_corporations['County']=active_corporations['County'].map(lambda x: format_county(x))
active_corporations=active_corporations[active_corporations['County'].map(lambda x: x in county_names)]

## Making dictionary to convert area to county

In [5]:
def change(x):
    if x=='bronx, kings, new york, queens, and richmond':
        return 'new york'
    else:
        return x

In [6]:
#First we make a dataframe with all the pairs of area and county
towns=pd.read_csv(r'towns.csv')
towns['Area']=towns['Town'].apply(lambda x: x.lower())

cities=pd.read_csv(r'cities.csv')
cities['Area']=cities['City'].apply(lambda x: x.lower())
cities['County']=cities['County'].apply(lambda x:change(x.lower()))#change to name of new york

city_pairs=cities[['Area','County']]
town_pairs=towns[['Area','County']]
pairs=city_pairs.append(town_pairs)
pairs['County']=pairs['County'].apply(lambda x:x.lower())
unique_pairs=pairs.drop_duplicates(subset=['Area','County'])
len(unique_pairs)

972

In [7]:
#now we deal with areas that are associated with multiple counties
# counted_areas=unique_pairs.groupby('Area').size().reset_index(name='counts')
# dups=list(counted_areas[counted_areas['counts']>1]['Area'])
# duplicate_areas=unique_pairs[unique_pairs['Area'].apply(lambda x: x in dups)]
# duplicate_areas.sort_values(by='Area')

In [8]:
areas_list=list(unique_pairs['Area'])
counties_list=list(unique_pairs['County'])
pair_dict={areas_list[i].lower() : counties_list[i].lower() for i in range(len(areas_list))}

In [9]:
#add ones that weren't matched based on name
pair_dict['spring valley village']='rockland'
pair_dict['rockville centre']='nassau'
pair_dict['de witt town']='onondaga'
pair_dict['lindenhurst']='suffolk'
pair_dict['freeport village']='nassau'
pair_dict['valley stream']='nassau'
pair_dict['richmond']='new york'

In [10]:
areas=unemployment['Area'].map(lambda x:format_county(x)).unique()
area2county={}
for area in areas:
    found=False
    for county in county_names:
        if found:
            break
        if area in county:
            val=county 
            found=True
            break
    if not found:
        for key in pair_dict.keys():
            if key in area or area in key:
                found=True
                val=pair_dict[key]
                break
    if found:
        area2county[area]=val
    else:
        print('not found:',area)
print('found %s/%s' % (str(len(area2county)),str(len(areas))))

not found: balance of state
not found: capital region
not found: finger lakes
not found: long island
not found: north country
not found: southern tier
found 198/204


## Cleaning unemployment dataset

In [11]:
unemployment=unemployment[unemployment['Area'].map(lambda x: format_county(x) in area2county )]
unemployment['County']=unemployment['Area'].map(lambda x: area2county[format_county(x)])

In [20]:
unemployment[unemployment['Area']=='Greene County']

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate,County
26423,Greene County,2021,2,20500,19000,1500,7.2,greene
26424,Greene County,2021,1,20700,19400,1400,6.7,greene
26425,Greene County,2020,12,20600,19400,1300,6.2,greene
26426,Greene County,2020,11,20000,18800,1200,5.9,greene
26427,Greene County,2020,10,20000,18800,1100,5.7,greene
...,...,...,...,...,...,...,...,...
26792,Greene County,1990,5,20500,19500,1000,4.9,greene
26793,Greene County,1990,4,20000,18900,1100,5.4,greene
26794,Greene County,1990,3,20900,19800,1200,5.6,greene
26795,Greene County,1990,2,21300,20000,1300,5.9,greene


In [22]:
unemployment.rename(columns=lambda x: x.strip() ,inplace=True)

In [23]:
unemployment.to_csv('unemployment.csv')

## Merging Datasets

In [13]:
active_corporations.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,Registered Agent Zip,Location Name,Location Address 1,Location Address 2,Location City,Location State,Location Zip,month,day,year
0,3211809,00:02:59 LLC,05/31/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,00:02:59 LLC,"656 UNION STREET, APT. 2",NaN,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,31,2005
1,4472700,000 LLC,10/15/2013,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,000,304 MESEROLE ST #2A,NaN,BROOKLYN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,15,2013
2,4723823,"000 MEADOW LANE, LLC",03/11/2015,suffolk,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O ACKERMAN O'BRIEN PACHMAN & BROWN LLP,34 PANTIGO ROAD,NaN,EAST HAMPTON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,11,2015
3,5224240,00168 HOME CORP.,10/26/2017,suffolk,NEW YORK,DOMESTIC BUSINESS CORPORATION,00168 HOME CORP.,28 PHEASANT RUN LANE,NaN,DIX HILLS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,26,2017
4,5581065,"0016 HEMPSTEAD TURNPIKE, LLC",07/02/2019,suffolk,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"GOLDENBOCK, EISEMAN, ASSOR, BELL & PESKOE LLP",ATTN LARRY A WELCH ESQ,711 3RD AVE FL 16,NEW YORK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2,2019


In [24]:
active_corporations.rename(columns=lambda x: x.strip() ,inplace=True)
active_corporations.to_csv('active_corporations.csv')

In [14]:
joined_df=active_corporations.merge(unemployment,how='inner',left_on=['County','month','year'],right_on=['County','Month','Year'])

In [15]:
joined_df.head()

,DOS ID,Current Entity Name,Initial DOS Filing Date,County,Jurisdiction,Entity Type,DOS Process Name,DOS Process Address 1,DOS Process Address 2,DOS Process City,...,month,day,year,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate
0,3211809,00:02:59 LLC,05/31/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,00:02:59 LLC,"656 UNION STREET, APT. 2",NaN,BROOKLYN,...,5,31,2005,Kings County,2005,5,1061300,999600,61700,5.8
1,3203583,1010 DEVELOPMENT LLC,05/11/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,C/O CHARALABOS P BAKALIS,8721 5TH AVE,SUITE 100,BROOKLYN,...,5,11,2005,Kings County,2005,5,1061300,999600,61700,5.8
2,3199867,103 MESEROLE LLC,05/03/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,103 MESEROLE LLC,107 CLYMER STREET,NaN,BROOKLYN,...,5,3,2005,Kings County,2005,5,1061300,999600,61700,5.8
3,3211699,"1059 E. 35TH STREET, LLC",05/31/2005,kings,NEW YORK,DOMESTIC LIMITED LIABILITY COMPANY,"1059 E. 35TH STREET, LLC",PO BOX 150-260,NaN,BROOKLYN,...,5,31,2005,Kings County,2005,5,1061300,999600,61700,5.8
4,3212110,"105 KINGS HIGHWAY CONDOMINIUM ASSOCIATION, INC.",05/31/2005,kings,NEW YORK,DOMESTIC NOT-FOR-PROFIT CORPORATION,"105 KINGS HIGHWAY CONDOMINIUM ASSOCIATION, INC.",SUITE 711,45 JOHN STREET,NEW YORK,...,5,31,2005,Kings County,2005,5,1061300,999600,61700,5.8


In [16]:
joined_df.shape

(4751620, 40)

In [17]:
active_corporations.shape

(852626, 33)

# Load

In [18]:
joined_df.to_csv('joined_dataset.csv')